## Predicting heart disease using machine learning
This notebook looks into using various Python-based machine learning and data science libraries in an attempt to 
build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

We're going to take the following approach:

   1. Problem definition
   2. Data
   3. Evaluation
   4. Features
   5. Modelling
   6. Experimentation
   
 ### 1. Problem Definition

In a statement,

    Given clinical parameters about a patient, can we predict whether or not they have heart disease?

### 2. Data

The original data came from the Cleavland data from the UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/datasets/heart+Disease

There is also a version of it available on Kaggle. https://www.kaggle.com/datasets/sumaiyatasmeem/heart-disease-classification-dataset

### 3. Evaluation

    If we can reach 95% accuracy at predicting whether or not a patient has heart disease during the proof of concept, we'll pursue the project.

###  4. Features

This is where you'll get different information about each of the features in your data. You can do this via doing your own research (such as looking at the links above) or by talking to a subject matter expert (someone who knows about the dataset).

Create data dictionary

    1. age - age in years
    2.  sex - (1 = male; 0 = female)
    3. cp - chest pain type
        0: Typical angina: chest pain related decrease blood supply to the heart
        1: Atypical angina: chest pain not related to heart
        2: Non-anginal pain: typically esophageal spasms (non heart related)
        3: Asymptomatic: chest pain not showing signs of disease
    4. restbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause              for concern
    5. chol - serum cholestoral in mg/dl
         1. serum = LDL + HDL + .2 * triglycerides 
         2. above 200 is cause for concern
    6. fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
         1. '>126' mg/dL signals diabetes
    7. restecg - resting electrocardiographic results
        
        0: Nothing to note
        1: ST-T Wave abnormality
            can range from mild symptoms to severe problems
            signals non-normal heart beat
        2: Possible or definite left ventricular hypertrophy
            Enlarged heart's main pumping chamber
    8. thalach - maximum heart rate achieved
    9. exang - exercise induced angina (1 = yes; 0 = no)
    10.oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during 
       exercise unhealthy heart will stress more
    11.slope - the slope of the peak exercise ST segment
        0: Upsloping: better heart rate with excercise (uncommon)
        1: Flatsloping: minimal change (typical healthy heart)
        2: Downslopins: signs of unhealthy heart
    12. ca - number of major vessels (0-3) colored by flourosopy
        1. colored vessel means the doctor can see the blood passing through 
        2. the more blood movement the better (no clots)
    13. thal - thalium stress result
        1,3: normal
        6: fixed defect: used to be defect but ok now
        7: reversable defect: no proper blood movement when excercising
    14. target - have disease or not (1=yes, 0=no) (= the predicted attribute)



##  Preparing the tools

We're going to use pandas, Matplotlib and NumPy for data analysis and manipulation.


In [1]:
# import all the tools we need

 #Regular EDA (exploratory data analysis) and plotting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# we want our plots to appear inside the notebook
%matplotlib inline 

# Models from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


# Model Evaluations
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

ImportError: cannot import name 'plot_roc_curve' from 'sklearn.metrics' (/home/angela/anaconda3/lib/python3.11/site-packages/sklearn/metrics/__init__.py)

## Load Data

In [ ]:
df=pd.read_csv("heart-disease.csv")
df

In [ ]:
# check shape of data
df.shape


## Data Exploration (exploratory data analysis or EDA)

The goal here is to find out more about the data and become a subject matter export on the dataset you're working with.

    What question(s) are you trying to solve?
    What kind of data do we have and how do we treat different types?
    What's missing from the data and how do you deal with it?
    Where are the outliers and why should you care about them?
    How can you add, change or remove features to get more out of your data?



In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#find out how many of each class there are in the labels column
df["target"].value_counts()

In [ ]:
df["target"].value_counts().plot(kind="bar",color=["salmon","lightblue"]);

In [ ]:
df.info();

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

### Heart Disease Frequency according to Sex

In [ ]:
df.sex.value_counts()

In [ ]:
# Compare target column with sex column
pd.crosstab(df.target,df.sex)

In [ ]:
# Create a plot of crosstab
pd.crosstab(df.target,df.sex).plot(kind="bar",figsize=(10,5),color=["salmon","lightblue"]);
plt.title("Heart Disease Frequency for Sex")
plt.xlabel("0 = No Disease, 1 = Disease")
plt.ylabel("Amount")
plt.legend(["Female","Male"])
plt.xticks(rotation=0);

## Age vs. Max Heart Rate for Heart Disease

In [ ]:
# create another figure
plt.figure(figsize=(10,6))

# scatter with positive examples
# fetching a subset of data where target is 1 in the age row
plt.scatter(df.age[df.target==1],
           df.thalach[df.target==1],
           c="salmon");

#scatter with negative examples
plt.scatter(df.age[df.target==0],
           df.thalach[df.target==0],
           c="lightblue");
# Add some helpful info
plt.title("Heart Disease in function of Age and Max Heart Rate")
plt.xlabel("Age")
plt.ylabel("Max Heart Rate")
plt.legend(["Disease", "No Disease"]);

In [ ]:
# Check the distribution of the age column with a histogram(the spread of the age)
df["age"].plot(kind="hist");
plt.title("Age Distribution")
plt.xlabel("Age Group")
plt.ylabel("Age")

## Heart Disease Frequency per Chest Pain Type

    cp - chest pain type
        0: Typical angina: chest pain related decrease blood supply to the heart
        1: Atypical angina: chest pain not related to heart
        2: Non-anginal pain: typically esophageal spasms (non heart related)
        3: Asymptomatic: chest pain not showing signs of disease



In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
# Make the crosstab more visual
pd.crosstab(df.cp, df.target).plot(kind="bar",
                                   figsize=(10, 6),
                                   color=["salmon", "lightblue"])

# Add some communication
plt.title("Heart Disease Frequency Per Chest Pain Type")
plt.xlabel("Chest Pain Type")
plt.ylabel("Amount")
plt.legend(["No Disease", "Disease"])
plt.xticks(rotation=0);

In [ ]:
# Make a correlation matrix
df.corr()

In [ ]:
# Let's make our correlation matrix a little prettier
corr_matrix=df.corr()
fig,ax=plt.subplots(figsize=(15,10))
ax=sns.heatmap(corr_matrix,
              annot=True,
              linewidths=0.5,
              fmt=".2f",
              cmap="YlGnBu")

## 5. Modelling

In [ ]:
df.head()

In [ ]:
#split data into X and y

X=df.drop("target",axis=1)
y=df["target"]


In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Split data into train and test sets
np.random.seed(42)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train

In [ ]:
y_train,len(y_train)

Now we've got our data split into training and test sets, it's time to build a machine learning model.

We'll train it (find the patterns) on the training set.

And we'll test it (use the patterns) on the test set.

We're going to try 3 different machine learning models:

    Logistic Regression
    K-Nearest Neighbours Classifier
    Random Forest Classifier



In [ ]:
#put models in a dictionary
models={
     "LogisticRegression":LogisticRegression(),
     "KNN": KNeighborsClassifier(),
     "Random Forest":RandomForestClassifier()
 }
# Create a function to fit and score models
def fit_and_score(models,X_train,X_test,y_train,y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of different Scikit-Learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    #set random seed
    np.random.seed(42)
    # make a dictionary to keep model scores
    model_scores={}
    # loop through the items in the models dictionary
    for name,model in models.items():
        #fit the model to the data
        model.fit(X_train,y_train);
        # evaluate the model and append the score to model score
        model_scores[name]=model.score(X_test,y_test)
    return model_scores


In [ ]:
model_scores=fit_and_score(models=models,
                          X_train=X_train,
                          X_test=X_test,
                          y_train=y_train,
                          y_test=y_test)
model_scores


## Model Comparison

In [ ]:
model_compare=pd.DataFrame(model_scores,index=["accuracy"])
model_compare.T.plot.bar();

Now we've got a baseline model... and we know a model's first predictions aren't always what we should based our next steps off. What should we do?

Let's look at the following:

    Hyperparameter tuning
    Feature importance
    Confusion matrix
    Cross-validation
    Precision
    Recall
    F1 score
    Classification report
    ROC curve
    Area under the curve (AUC)


## Hyperparameter tuning (by hand)

In [ ]:
# Let's tune KNN
train_scores=[]
test_scores=[]

# Create a list of differnt values for n_neighbors
neighbors=range(1,21)

# Setup KNN instance
knn=KNeighborsClassifier()


# Loop through different n_neighbors
for i in neighbors:
    knn.set_params(n_neighbors=i)
    
    # fit the algorithm
    knn.fit(X_train,y_train)
    # Update the training scores list
    train_scores.append(knn.score(X_train,y_train))  
    # Update the test scores list
    test_scores.append(knn.score(X_test,y_test))

In [ ]:
train_scores

In [ ]:
test_scores

In [ ]:
plt.plot(neighbors,train_scores,label="Train score")
plt.plot(neighbors,test_scores,label="Test score");
plt.xlabel("No of neighbors")
plt.ylabel("Model Score")
plt.xticks(np.arange(1,21,1))
plt.legend()
print(f"Maximum KNN score on the test data: {max(test_scores)*100:.2f}%")

## Hyperparameter tuning with RandomizedSearchCV

We're going to tune:

    LogisticRegression()
    RandomForestClassifier()

... using RandomizedSearchCV


In [ ]:
# Create a hyperparameter grid for LogisticRegression
log_reg_grid={"C":np.logspace(-4,4,20),
             "solver":["liblinear"]}
# Create a hyperparameter grid for RandomForestClassifier
rf_grid={"n_estimators":np.arange(10,1000,50),
        "max_depth":[None,3,5,10],
        "min_samples_split":np.arange(2,20,2),
        "min_samples_leaf":np.arange(1,20,2)}

Now we've got hyperparameter grids setup for each of our models, let's tune them using RandomizedSearchCV...

In [ ]:
# Tune LogisticRegression
np.random.seed(42)
# Setup random hyperparameter search for LogisticRegression
rs_log_reg=RandomizedSearchCV(LogisticRegression(),
                             param_distributions=log_reg_grid,
                             cv=5,
                             n_iter=20,
                             verbose=2)
# Fit random hyperparameter search model for LogisticRegression
rs_log_reg.fit(X_train,y_train)




In [ ]:
#check the best params
rs_log_reg.best_params_

In [ ]:
#Evaluate the model with the new paramereter existing
rs_log_reg.score(X_test,y_test)

Now we've tuned LogisticRegression(), let's do the same for RandomForestClassifier().

In [ ]:
#setup random seed()
np.random.seed(42)
# Setup random hyperparameter search for RandomForestClassifier
rs_rf=RandomizedSearchCV(RandomForestClassifier(),
                    param_distributions=rf_grid,
                    cv=5,
                    n_iter=20,
                    verbose=True)
rs_rf.fit(X_train,y_train)

In [ ]:
# Find the best hyperparameters
rs_rf.best_params_

In [ ]:
# Evaluate the randomized search RandomForestClassifier model
rs_rf.score(X_test,y_test)


## Hyperparamter Tuning with GridSearchCV

Since our LogisticRegression model provides the best scores so far, we'll try and improve them again using GridSearchCV...


In [ ]:
# Different hyperparameters for our LogisticRegression model
log_reg_grid = {"C": np.logspace(-4, 4, 30),
                "solver": ["liblinear"]}
# Setup grid hyperparameter search for LogisticRegression
gs_log_reg=GridSearchCV(LogisticRegression(),
                       param_grid=log_reg_grid,
                       cv=5,
                       verbose=2)
# Fit grid hyperparameter search model
gs_log_reg.fit(X_train, y_train);

In [ ]:
gs_log_reg.best_params_

In [ ]:
# Evaluate the grid search LogisticRegression model
gs_log_reg.score(X_test, y_test)

In [ ]:
model_scores

In [ ]:
import pickle
pickle.dump(gs_log_reg,open("gs_log_reg_model_5.pkl","wb"))

In [ ]:
loaded_model=pickle.load(open("gs_log_reg_model_5.pkl","rb"))

In [ ]:
loaded_model.predict(X_test)

In [ ]:
loaded_model.score(X_test,y_test)

In [ ]:
loaded_model.predict(X_test)


Evaluting our tuned machine learning classifier, beyond accuracy

    ROC curve and AUC score
    Confusion matrix
    Classification report
    Precision
    Recall
    F1-score

... and it would be great if cross-validation was used where possible.

To make comparisons and evaluate our trained model, first we need to make predictions.


In [ ]:
# Make predictions with tuned model
y_preds=gs_log_reg.predict(X_test)

In [ ]:
y_preds

In [ ]:
y_test

In [ ]:
# Plot ROC curve and calculate and calculate AUC metric
plot_roc_curve(gs_log_reg,X_test,y_test);

In [ ]:
# Confusion matrix
print(confusion_matrix(y_test, y_preds))

## Creating a confusion matrix using Scikit-Learn
Scikit-Learn has multiple different implementations of plotting confusion matrices:
1.	sklearn.metrics.ConfusionMatrixDisplay.from_estimator(estimator, X, y) - this takes a fitted estimator (like our clf model), features (X) and labels (y), it then uses the trained estimator to make predictions on X and compares the predictions to y by displaying a confusion matrix.
2.	sklearn.metrics.ConfusionMatrixDisplay.from_predictions(y_true, y_pred) - this takes truth labels and predicted labels and compares them by displaying a confusion matrix.


In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true=y_test, 
                                   y_pred=y_preds);


Now we've got a ROC curve, an AUC metric and a confusion matrix, let's get a classification report as well as cross-validated precision, recall and f1-score.

In [ ]:
print(classification_report(y_test,y_preds))

## Calculate evaluation metrics using cross-validation

We're going to calculate accuracy, precision, recall and f1-score of our model using cross-validation and to do so we'll be using cross_val_score().


In [ ]:
#check best hyperparameters
gs_log_reg.best_params_


In [ ]:
#Create a new classifier with best parameters
clf = LogisticRegression(C=0.20433597178569418,
                         solver="liblinear")

In [ ]:
# Cross-validated accuracy
cv_acc=cross_val_score(clf,
                      X,
                      y,
                      cv=5,
                      scoring="accuracy")
cv_acc

In [ ]:
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
# Cross-validated precision
cv_pres=cross_val_score(clf,
                      X,
                      y,
                      cv=5,
                      scoring="precision")
cv_pres

In [ ]:
cv_pres=np.mean(cv_pres)
cv_pres

In [ ]:
# Cross-validated recall
cv_rec=cross_val_score(clf,
                      X,
                      y,
                      cv=5,
                      scoring="recall")
cv_rec

In [ ]:
cv_rec=np.mean(cv_rec)
cv_rec

In [ ]:
# Cross-validated f1 score
cv_f1=cross_val_score(clf,
                      X,
                      y,
                      cv=5,
                      scoring="f1")
cv_f1

In [ ]:
cv_f1=np.mean(cv_f1)
cv_f1

In [ ]:
# Visualize cross-validated metrics
# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({"Accuracy": cv_acc,
                           "Precision": cv_pres,
                           "Recall": cv_rec,
                           "F1": cv_f1},
                          index=[0])

cv_metrics.T.plot.bar(title="Cross-validated classification metrics",
                      legend=False);

## Feature Importance

Feature importance is another as asking, "which features contributed most to the outcomes of the model and how did they contribute?"

Finding feature importance is different for each machine learning model. One way to find feature importance is to search for "(MODEL NAME) feature importance".

Let's find the feature importance for our LogisticRegression model...


In [ ]:
# Fit an instance of LogisticRegression
clf = LogisticRegression(C=0.20433597178569418,
                         solver="liblinear")

clf.fit(X_train, y_train);

In [ ]:
# Its an attribute of the logistic regression model to find how each feature contributes to the overall target result 
clf.coef_

In [ ]:
# Match coef's of features to columns
feature_dict=dict(zip(df.columns,list(clf.coef_[0])))
feature_dict

In [ ]:
# Visualize feature importance
feature_df = pd.DataFrame(feature_dict, index=[0])
feature_df.T.plot.bar(title="Feature Importance", legend=False);

In [ ]:
pd.crosstab(df["sex"], df["target"])

In [ ]:
pd.crosstab(df["slope"], df["target"])


slope - the slope of the peak exercise ST segment

    0: Upsloping: better heart rate with excercise (uncommon)
    1: Flatsloping: minimal change (typical healthy heart)
    2: Downslopins: signs of unhealthy heart



## 6. Experimentation

If you haven't hit your evaluation metric yet... ask yourself...

    Could you collect more data?
    Could you try a better model? Like CatBoost or XGBoost?
    Could you improve the current models? (beyond what we've done so far)
    If your model is good enough (you have hit your evaluation metric) how would you export it and share it with others?

